#### Extraction of keycode from mdb's into excel sheets
* Extraction of keycode and gps in excel file from mdb's
* Extraction of distress in excel file from keycode
* Extraction of remarks in excel file from keycode
* Converting distress excel into table
* Creating point and area distress's table from distress table
* Creating point and line feature classes of distress's
* Reprojection of created feature classes into UTM coordinates reference system 


In [5]:
import pyodbc,os,pandas as pd
arcpy.env.workspace = r"D:\_1.Project\AACM\Pavement\GDB\Distress Data Extraction.gdb"
mdb_directory = r"D:\_1.Project\AACM\Pavement\Data\DataView MDB's"
o_gps = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Distress_Workbook\E_GPS.xlsx"
o_keycode = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Distress_Workbook\E_Keycode.xlsx"
o_distress = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Distress_Workbook\E_distress_keycode.xlsx"
o_distress_remarks = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Distress_Workbook\E_distress_Remarks.xlsx"
#defining function to read mdb's
def read_mdb(x):
    mdb_files = []
    for root, dirs, files in os.walk(x):
        for file in files:
            if file.endswith(".mdb"):
                mdb_files.append(os.path.join(root, file))
    return mdb_files

#defining function to read tables in mdb
def read_tables_by_prefix(mdb_file, table_prefix):
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};Dbq={mdb_file};"
    connection = pyodbc.connect(conn_str)
    cursor = connection.cursor()

    tables = []
    for table_info in cursor.tables(tableType='TABLE'):
        table_name = table_info.table_name
        if table_name.startswith(table_prefix):
            tables.append(table_name)

    data = pd.DataFrame()
    for table in tables:
        query = f"SELECT * FROM [{table}]"
        table_data = pd.read_sql(query, connection)
        table_data['Survey_ID_1'] = os.path.splitext(os.path.basename(mdb_file))[0]
        data = pd.concat([data, table_data], ignore_index=True)

    connection.close()
    return data


#calling function to read mdb's
mdb_files = read_mdb(mdb_directory)
print(len(mdb_files))


#pandas data frame for storing extracted gps and keycode
GPS_P = pd.DataFrame()
DV_keycode = pd.DataFrame()


#extracting and merging gps and keycode from mdb's 
for mdb_file in mdb_files:
    dt_vedio = read_tables_by_prefix(mdb_file, "dt_VideoKeyCode_Raw")
    gps = read_tables_by_prefix(mdb_file, "dt_GPS_Processed")

    DV_keycode = pd.concat([DV_keycode, dt_vedio], ignore_index=True)
    GPS_P = pd.concat([GPS_P, gps], ignore_index=True)

    
#filtering data frame to remove calibration rows from pandas data frame    
GPS_P_filtered = GPS_P[~GPS_P['Survey_ID'].str.startswith('CALIBRATION')]
DV_keycode_filtered = DV_keycode[~DV_keycode['SURVEY_ID'].str.startswith('CALIBRATION')]


#exporting gps and keycode from pandas data frame
GPS_P_filtered.to_excel(o_gps, index=False)
DV_keycode_filtered.to_excel(o_keycode, index = False)


#creating two data frame for distress and remarks
field = 'EVENT_DESC'
field_values = ['REMARKS','PAVEMENT MARKING']
distress = DV_keycode[~DV_keycode[field].isin(field_values)]
distress_Remark = DV_keycode[DV_keycode[field].isin(field_values)]

#exporting distress and remarks excel from pandas data frame
distress.to_excel(o_distress, index = False)
distress_Remark.to_excel(o_distress_remarks, index = False) 

#converting distress and remarks excel workbooks into tables
arcpy.ExcelToTable_conversion(r"D:\_1.Project\AACM\Pavement\Excel Workbook\Distress_Workbook\E_distress_keycode.xlsx", "Keycode_tbl", "Sheet1")
arcpy.ExcelToTable_conversion(r"D:\_1.Project\AACM\Pavement\Excel Workbook\Distress_Workbook\E_distress_Remarks.xlsx", "Remarks_tbl", "Sheet1")

#creating point distress by select by attribute method, where chainage start and chainage end are equal
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "NEW_SELECTION", '"CHAINAGE_START" = "CHAINAGE_END" ')
arcpy.TableToTable_conversion("Keycode_tbl", arcpy.env.workspace, "P_Distress")
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "CLEAR_SELECTION")

#creating Line distress by select by attribute method, where chainage start and end is not equal
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "NEW_SELECTION", '"CHAINAGE_START" <> "CHAINAGE_END" ')
arcpy.TableToTable_conversion("Keycode_tbl", arcpy.env.workspace, "L_Distress")
arcpy.management.SelectLayerByAttribute("Keycode_tbl", "CLEAR_SELECTION")


#creating feature point and line classes from point and line distess tables 
arcpy.management.XYTableToPoint("P_Distress","Distress_P_wgs","X","Y","",arcpy.SpatialReference(4326))
arcpy.management.XYToLine("L_Distress","Distress_L_wgs","X","Y","X_END","Y_END","GEODESIC","",arcpy.SpatialReference(4326),True)

utm_crs = arcpy.SpatialReference(32640)
arcpy.Project_management("Distress_P_wgs","P_distress_utm",utm_crs)
arcpy.Project_management("Distress_L_wgs","L_distress_utm",utm_crs)


print(GPS_P_filtered.head(10))
print("Successfully completed!!")


57
                  Survey_ID  Chainage      SPEED  LRP_Number  LRP_OFFSET  \
1159  CL_ADI_119_5_RO5-TL-1       0.0  65.500000           0         0.0   
1160  CL_ADI_119_5_RO5-TL-1       1.0  65.500000           0         1.0   
1161  CL_ADI_119_5_RO5-TL-1       2.0  65.500000           0         2.0   
1162  CL_ADI_119_5_RO5-TL-1       3.0  65.500000           0         3.0   
1163  CL_ADI_119_5_RO5-TL-1       4.0  65.500000           0         4.0   
1164  CL_ADI_119_5_RO5-TL-1       5.0  65.500000           0         5.0   
1165  CL_ADI_119_5_RO5-TL-1       6.0  65.500000           0         6.0   
1166  CL_ADI_119_5_RO5-TL-1       7.0  65.400002           0         7.0   
1167  CL_ADI_119_5_RO5-TL-1       8.0  65.400002           0         8.0   
1168  CL_ADI_119_5_RO5-TL-1       9.0  65.400002           0         9.0   

          GPS_TIME   Latitude  Longitude   Northing    Easting  Alt_HAE  \
1159  07:19:34.651  24.467646  54.395432  24.467646  54.395432      0.0   
1160  07:1

### The below script will follow after snapping of distress line to pms sections
* first, start and end vertexs of pms section will produced which used for splitting line distresses
* lines distress will under go for splitting based on points produced from pms sections at start and end vertexes
* Quantity value will be calculated

In [ ]:
from collections import Counter

arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True

arcpy.management.AddFields("L_distress_utm",[["section_occurence","TEXT","",300],["split_len","TEXT"],["f_part","TEXT"],
                                               ["s_part","TEXT"],["qc","TEXT"],["Quantity","DOUBLE"],["Section_len","DOUBLE"]])

arcpy.management.AddFields("P_distress_utm",[["section_occurence","TEXT","",300],["split_len","TEXT"],[ "f_part","TEXT"],
                                               ["s_part","TEXT"],["qc","TEXT"],["Quantity","DOUBLE"],["Section_len","DOUBLE"]])    


arcpy.management.CalculateField("L_distress_utm","Section_len", "!SHAPE.LENGTH!", "PYTHON3")

expr = f"!SURVEY_ID!+!COMMENT_!+!COMMENT_1!+!EVENT_DESC!+str(!CHAINAGE_START!)+str(!CHAINAGE_END!)+str(!LENGTH!)"
 
arcpy.management.CalculateField("L_distress_utm","section_occurence", expr, "PYTHON3")

expr1 = f"!SURVEY_ID!+!COMMENT_1!+!EVENT_DESC!+str(!CHAINAGE_START!)+str(!CHAINAGE_END!)+str(!LENGTH!)"
arcpy.management.CalculateField("P_distress_utm","section_occurence", expr1, "PYTHON3")

arcpy.management.FeatureVerticesToPoints("PMS_Sections","PMS_sec_Points","BOTH_ENDS")
arcpy.management.SplitLineAtPoint("L_distress_utm","L_GPS_Points", "Distress_L_split","0.5 Meters")



def multiply_fields(a, b, c, d):
    try:
        a_value = float(a) if a is not None else 1.0
        b_value = float(b) if b is not None else 1.0
        c_value = float(c) if c is not None else 1.0
        d_value = float(d) if d is not None else 1.0
        result = a_value * b_value * c_value * d_value
        return result
    except (ValueError,TypeError):
        return None
    

def multiply_p_distress(a, b, c):
    try:
        a_value = float(a) if a is not None else 1.0
        b_value = float(b) if b is not None else 1.0
        c_value = float(c) if c is not None else 1.0
        result = a_value * b_value * c_value
        return result
    except (ValueError,TypeError):
        return None
    
# Use a Search Cursor to count occurrences of each distress_L
section_counter = Counter()
with arcpy.da.SearchCursor("Distress_L_split", ["section_occurence"]) as cursor:
    for row in cursor:
        section_counter[row[0]] += 1


with arcpy.da.UpdateCursor("Distress_L_split", ["section_occurence", 'SHAPE@', "COMMENT_1", "split_len", "f_part",
                                       "s_part","qc","Quantity","Section_len"]) as cursor:
    for row in cursor:
        same_sections = row[0]
        if section_counter[same_sections] > 1 and '*' not in row[2]:      # distress length fall more than one section for linear distress
            row[7] = (((row[1].length)/row[8]) * (float(row[2])))         # fallen length/distressL * ShapeLength
        elif section_counter[same_sections] > 1 and '*' in row[2]:        # distress length fall in more than one section for area distress
            row[4] = row[2].split("*")[-1]                                # getting width of distress
            row[7] = float(row[4]) * row[1].length                        # splited distress lenght multiply by width
        elif section_counter[same_sections] == 1 and '*' not in row[2]:   # distress length fall in one section for linear distress
            row[7] = float(row[2])
        elif section_counter[same_sections] == 1 and '*' in row[2]:       # distress length fall in one section for area distress
            row[4] = row[2].split("*")[0]
            row[5] = row[2].split("*")[-1]
            row[7] = float(row[4]) * float(row[5])  
        else:
            row[6] = "check the values"
        cursor.updateRow(row)

with arcpy.da.UpdateCursor("P_distress_utm", ["COMMENT_1", "split_len", "f_part", "qc", "Quantity", "Section_len"]) as cursor:
    for row in cursor:
        if "*" in row[0]:
            row[1] = row[0].split("*")[0]
            row[2] = row[0].split("*")[-1]
            row[4] = float(row[1]) * float(row[2])
        elif "*" not in row[0]:
            row[4] = float(row[0])
        else:
            row[3] = "check the values"
        cursor.updateRow(row)
     
print ("sucessfully completed")

### The below script will follow after splitting line distress's and calulation of quantity
* convertig distress_L into midpoints
* combine mid points of line distress and point distress to produced total or combine distresses
* spatial join one to many with L_GPS_sections will carried out
* a new field will be added for creating distress event desc and severity 


In [ ]:
arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True
arcpy.management.FeatureVerticesToPoints("Distress_L_split","Distress_L_midpoints","MID")

arcpy.management.Merge(["Distress_L_midpoints","P_distress_utm"],"Distress_Combine","",'ADD_SOURCE_INFO')
arcpy.analysis.SpatialJoin("L_GPS_Section","Distress_Combine","L_GPS_distress_Pts","JOIN_ONE_TO_MANY","KEEP_ALL","","INTERSECT","0.5 METERS")

fields = arcpy.ListFields("L_GPS_distress_Pts")
arcpy.management.AddField("L_GPS_distress_Pts","Event_distress","TEXT")


with arcpy.da.UpdateCursor("L_GPS_distress_Pts",["EVENT_DESC","COMMENT","Event_distress","COMMENT_"]) as cursor:
    for row in cursor:
        if row[0]  == "ALLIGATOR CRACKING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F1_L_Qty"
        elif row[0] == "ALLIGATOR CRACKING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F1_M_Qty"
        elif row[0] == "ALLIGATOR CRACKING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F1_H_Qty"
        elif row[0] == "BLEEDING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F2_L_Qty"
        elif row[0] == "BLEEDING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F2_M_Qty"
        elif row[0] == "BLEEDING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F2_H_Qty"
        elif row[0] == "BLOCK CRACKING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F3_L_Qty"
        elif row[0] == "BLOCK CRACKING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F3_M_Qty"
        elif row[0] == "BLOCK CRACKING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F3_H_Qty"
        elif row[0] == "BUMPS & SAGS" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F4_L_Qty"
        elif row[0] == "BUMPS & SAGS" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F4_M_Qty"
        elif row[0] == "BUMPS & SAGS" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F4_H_Qty"
        elif row[0] == "CORRUGATION"  and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F5_L_Qty"
        elif row[0] == "CORRUGATION" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F5_M_Qty"
        elif row[0] == "CORRUGATION" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F5_H_Qty"
        elif row[0] == "DEPRESSION" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F6_L_Qty"
        elif row[0] == "DEPRESSION" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F6_M_Qty"
        elif row[0] == "DEPRESSION" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F6_H_Qty"
        elif row[0] == "EDGE CRACKING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F7_L_Qty"
        elif row[0] == "EDGE CRACKING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F7_M_Qty"
        elif row[0] == "EDGE CRACKING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F7_H_Qty"
        elif row[0] == "JT REFLECTION CRACKING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F8_L_Qty"
        elif row[0] == "JT REFLECTION CRACKING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F8_M_Qty"
        elif row[0] == "JT REFLECTION CRACKING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F8_H_Qty"
        elif row[0] == "LANE SHOULDER DROP OFF" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F9_L_Qty"
        elif row[0] == "LANE SHOULDER DROP OFF" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F9_M_Qty"
        elif row[0] == "LANE SHOULDER DROP OFF" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F9_H_Qty"
        elif row[0] == "LONG & TRANS CRACKING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F10_L_Qty"
        elif row[0] == "LONG & TRANS CRACKING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F10_M_Qty"
        elif row[0] == "LONG & TRANS CRACKING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F10_H_Qty"
        elif row[0] == "PATCHING & UTILITY CUT PATCHING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F11_L_Qty"
        elif row[0] == "PATCHING & UTILITY CUT PATCHING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F11_M_Qty"
        elif row[0] == "PATCHING & UTILITY CUT PATCHING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F11_H_Qty"
        elif row[0] == "POLISHED AGGREGATE" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F12_L_Qty"
        #elif row[0] == "POLISHED AGGREGATE" and row[1] == "MEDIUM":
            row[2] = "F12_M_Qty"
        #elif row[0] == "POLISHED AGGREGATE" and row[1] == "HIGH":
            row[2] = "F12_H_Qty"
        elif row[0] == "POTHOLES" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F13_L_Qty"
        elif row[0] == "POTHOLES" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F13_M_Qty"
        elif row[0] == "POTHOLES" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F13_H_Qty"
        elif row[0] == "RAILROAD CROSSING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F14_L_Qty"
        elif row[0] == "RAILROAD CROSSING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F14_M_Qty"
        elif row[0] == "RAILROAD CROSSING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F14_H_Qty"
        elif row[0] == "RUTTING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F15_L_Qty"
        elif row[0] == "RUTTING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F15_M_Qty"
        elif row[0] == "RUTTING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F15_H_Qty"
        elif row[0] == "SHOVING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F16_L_Qty"
        elif row[0] == "SHOVING"  and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F16_M_Qty"
        elif row[0] == "SHOVING"  and  (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F16_H_Qty"
        elif row[0] == "SLIPPAGE CRACKING"  and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F17_L_Qty"
        elif row[0] == "SLIPPAGE CRACKING"  and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F17_M_Qty"
        elif row[0] == "SPLIPPAGE CRACKING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F17_H_Qty"
        elif row[0] == "UPHEAVIAL & SWELL" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F18_L_Qty"
        elif row[0] == "UPHEAVIAL & SWELL" and (row[1] == "MEDIUM" or row[3] ==  "MEDIUM"):
            row[2] = "F18_M_Qty"
        elif row[0] == "UPHEAVIAL & SWELL" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F18_H_Qty"
        elif row[0] == "RAVELLING" and (row[1] == "LOW" or row[3] == "LOW"):
            row[2] = "F19_M_Qty"
        elif row[0] == "RAVELLING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"):
            row[2] = "F19_M_Qty"
        elif row[0] == "RAVELLING" and (row[1] == "HIGH" or row[3] == "HIGH"):
            row[2] = "F19_H_Qty"
        elif row[0] == "WEATHERING" and (row[1] == "LOW" or row[3] == "LOW"): 
            row[2] = "F20_L_Qty"
        elif row[0] == "WEATHERING" and (row[1] == "MEDIUM" or row[3] == "MEDIUM"): 
            row[2] = "F20_M_Qty"
        elif row[0] == "WEATHERING" and (row[1] == "HIGH" or row[3] ==  "HIGH"):
            row[2] = "F20_H_Qty"
        else:
            row[2] = "remarks"
        cursor.updateRow(row)
print ("sucessfully completed!!!")


### The below script will run after merging mid point and mid points distress and preparation of template for pivot
* first, pivot table is created
* summary of pivot table for sum based on SC_UUID will produced

In [ ]:
arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
feature_class = "L_GPS_distress_Pts"
arcpy.management.PivotTable(feature_class,["OBJECTID","OBJECTID"],"Event_distress","Quantity","pvt_tbl")
list_field = []
tbl = arcpy.ListFields("pvt_tbl")
for field in tbl:
    if field.name.startswith("F"):
        list_field.append(field.name) 
if list_field:
    statistics_types = "SUM"
    output_tbl = r'E:\ADM\Project\mdb_processing\Distress_Integration.gdb\o_tbl_2'
    statistics_fields = ";".join([f"{field} {statistics_types}" for field in list_field])
    arcpy.analysis.Statistics("pvt_tbl", output_tbl, statistics_fields, "SC_UUID")

    print("Statistics analysis completed.")
else:
    print("No fields starting with 'F' found.")

### The below script will follow after production of summary table
* The output of this below step will forwarded for paver in the form of shapefile
* PMS sections on the template of paver will imported
* Join paver to summary table and caclulate distress's and remove join

In [ ]:
arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
feature_class = 'paver_file'
table = 'o_tbl_2'

# Define the common field
common_field = 'SC_UUID'

summary_fields = [field.name for field in arcpy.ListFields(table) if field.name.startswith("SUM")]
summary_to_paver = [field.replace("SUM_", "") for field in summary_fields]

# Define the fields to be calculated
fields_to_calculate = summary_to_paver

# Join the table to the feature class
arcpy.management.AddJoin(feature_class, common_field, table, common_field)

# Calculate fields in the feature class based on the corresponding fields in the table
for field in fields_to_calculate:
    expression = '!{}!'.format('o_tbl_2.SUM_' + field)
    arcpy.management.CalculateField(feature_class, field, expression, 'PYTHON3')

# Remove the join
arcpy.management.RemoveJoin(feature_class)

print("Fields calculated successfully.")


